# Debug

In [1]:
from functools import partial
from qiskit.quantum_info import SparsePauliOp, random_statevector

import scipy.sparse.linalg as ssla
from scipy import sparse
import pandas as pd
import scipy
import jax

import matplotlib.pyplot as plt
import numpy as np

from utils import *
# from lightcone import *
from spin_ham import *
from trotter import *
from bounds import *
# from noise import *

figs_dir, data_dir = './figs', './data'

In [ ]:
# demo of the nearest-neighbor Hamiltonian
hnn = Nearest_Neighbour_1d(3, Jx=2, Jy=2, Jz=2, hx=0, hy=0, hz=1, pbc=False, verbose=True)
n = 8
r = 100
J, h = 1, 0.2
t_list = np.linspace(1, 10, 20)

hnn = Nearest_Neighbour_1d(n, Jx=J, Jy=J, Jz=J, hx=0, hy=0, hz=h, pbc=False, verbose=False)
singl_ob = SparsePauliOp.from_sparse_list([('Z', [0], 1)], n).to_matrix()
multi_ob = SparsePauliOp.from_sparse_list([(random.choice(['X','Y','Z']), [i], 1) for i in range(0, n)], n).to_matrix()
multi_ob = multi_ob / np.linalg.norm(multi_ob, ord=2)
# multi_ob = SparsePauliOp.from_sparse_list([('Z', [i], 1/n) for i in range(0, n)], n).to_matrix()
h_list = [h.to_matrix(True) for h in hnn.ham_par]
for t in t_list:
    exact_U = scipy.linalg.expm(-1j * t * sum([h.to_matrix() for h in hnn.ham_par]))
    appro_U = pf_r(h_list, t, r, order=2)
    exact_ob_s = exact_U.conj().T @ singl_ob @ exact_U 
    appro_ob_s = appro_U.conj().T @ singl_ob @ appro_U
    exact_ob_m = exact_U.conj().T @ multi_ob @ exact_U 
    appro_ob_m = appro_U.conj().T @ multi_ob @ appro_U
    ob_error_s = np.linalg.norm(exact_ob_s - appro_ob_s, ord=2)
    ob_error_m = np.linalg.norm(exact_ob_m - appro_ob_m, ord=2)
    print('single ob error (operator norm): ', ob_error_s)
    print('multip ob error (operator norm): ', ob_error_m)

## Different norms

In [2]:
from spin_ham import *
from qiskit.quantum_info import SparsePauliOp
import scipy, numpy
from trotter import *
import sys

n = 9
r = 1
t = 0.5
J, h = 1, 0.2
hnn = Nearest_Neighbour_1d(n, Jx=0, Jy=0, Jz=J, hx=h, hy=0, hz=0, pbc=False, verbose=False)
ob = SparsePauliOp.from_sparse_list([('Z', [0], 1)], n).to_matrix()
exact_U = scipy.linalg.expm(-1j * t * sum([h.to_matrix() for h in hnn.ham_par]))
h_list = [h.to_matrix(True) for h in hnn.ham_par]
appro_U = pf_r(h_list, t, r, order=2)
exact_ob_s = exact_U.conj().T @ ob @ exact_U 
appro_ob_s = appro_U.conj().T @ ob @ appro_U
scipy.linalg.norm(exact_ob_s - appro_ob_s, ord=2)  
print(np.linalg.norm(exact_ob_s))
print(np.linalg.norm(appro_ob_s, ord=2))
print(np.linalg.norm(exact_ob_s - appro_ob_s, ord='nuc'))
print(np.trace(exact_ob_s - appro_ob_s))
print(np.linalg.norm(exact_ob_s - appro_ob_s, ord=2))
# np.linalg.eigvals(exact_ob_s - appro_ob_s)
print(np.sort(abs(np.linalg.eigvalsh(exact_ob_s - appro_ob_s)))[-1])
# exact_ob_m = exact_U.conj().T @ multi_ob @ exact_U 
# appro_ob_m = appro_U.conj().T @ multi_ob @ appro_U
file_path = 'matrix6.txt'
# Save the matrix to the file
np.savetxt(file_path, exact_ob_s - appro_ob_s, delimiter=',')
# sys.modules[__name__].__dict__.clear()
# globals().clear()
# %reset

22.627416997969508
1.0000000000000169
1.4649881645461555
(-9.015010959956271e-14+4.255092054469643e-18j)
0.0028613051827964587
0.002861305182796463


## Expm (JAX and Scipy)

In [38]:
# from spin_ham import *
# from trotter import *
# import sys
from qiskit.quantum_info import SparsePauliOp
import scipy
import numpy as np
import jax

t = 0.5
J, h = 1, 0.2
for n in [7, 8, 9, 10]:
# for n in [7, 8, 9, 10, 11]:
    # r = 1
    # hnn = Nearest_Neighbour_1d(n, Jx=0, Jy=0, Jz=J, hx=h, hy=0, hz=0, pbc=False, verbose=False)
    zz_tuples = [('ZZ', [i, i + 1], J) for i in range(0, n-1)]
    x_tuples = [('X', [i], h) for i in range(0, n)] 
    ham = SparsePauliOp.from_sparse_list([*zz_tuples, *x_tuples], n).simplify() 
    even_terms = SparsePauliOp.from_sparse_list([*zz_tuples[::2], *x_tuples[::2]], n).simplify()
    odd_terms = SparsePauliOp.from_sparse_list([*zz_tuples[1::2], *x_tuples[1::2]], n).simplify()
    ham_par = [even_terms, odd_terms]
    ob = SparsePauliOp.from_sparse_list([('Z', [0], 1)], n).to_matrix()
    # print('ob: ', ob)
    # exact_U = scipy.linalg.expm(-1j * t * sum([h.to_matrix() for h in hnn.ham_par]))
    h_list = [h.to_matrix() for h in ham_par]
    exact_U = np.matrix(jax.scipy.linalg.expm(-1j * t * sum([h for h in h_list])))
    print('n = ', n, h_list[0].shape)
    # for r in [1]:
    for r in [1, 2]:
        list_U = [jax.scipy.linalg.expm(-1j * (t / (2*r)) * h) for h in h_list]
        # list_U = [scipy.linalg.expm(-1j * (t / (2*r)) * h) for h in h_list]
        # if verbose: print('----expm Herm finished----')
        # appro_U_dt_forward = np.linalg.multi_dot(list_U)
        # appro_U_dt_reverse = np.linalg.multi_dot(list_U[::-1])
        appro_U_dt_forward = np.matmul(list_U[0], list_U[1])
        appro_U_dt_reverse = np.matmul(list_U[1], list_U[0])
        # appro_U_dt_forward = list_U[0] @ list_U[1]
        # appro_U_dt_reverse = list_U[1] @ list_U[0]
        # appro_U_dt = list_U[0] @ list_U[1]
        # if verbose: print('----matrix product finished----')
        # appro_U = np.matrix(appro_U_dt_reverse @ appro_U_dt_forward)
        appro_U = np.matrix(np.linalg.matrix_power(np.matmul(appro_U_dt_reverse, appro_U_dt_forward), r))
        # appro_U = np.matrix(np.linalg.matrix_power(appro_U_dt_reverse @ appro_U_dt_forward, r))
        # appro_U = np.matrix(pf_r(h_list, t, r, order=2))
        exact_ob = np.matmul(exact_U.H, np.matmul(ob, exact_U))
        appro_ob = np.matmul(appro_U.H, np.matmul(ob, appro_U))
        # exact_ob = exact_U.H @ ob @ exact_U 
        # appro_ob = appro_U.H @ ob @ appro_U
        np.allclose(exact_ob, exact_ob.conj().T)
        np.allclose(appro_ob, appro_ob.conj().T)
        # np.save(f'./data/exact_ob_{n}_{r}.npy', appro_U)

        # appro_ob_s = appro_U.conj().T @ ob @ appro_U
        # scipy.linalg.norm(exact_ob_s - appro_ob_s, ord=2)  
        # print(np.linalg.norm(exact_ob_s))
        # print(np.linalg.norm(appro_ob_s, ord=2))
        # print(np.linalg.norm(exact_ob_s - appro_ob_s, ord='nuc'))
        # print(np.trace(exact_ob_s - appro_ob_s))
        # print(np.linalg.norm(exact_ob - appro_ob, ord=2))
        err_op = exact_ob - appro_ob
        # np.linalg.eigvals(exact_ob_s - appro_ob_s)
        print(np.sort(abs(np.linalg.eigvalsh(err_op)))[-1])
        # print(exact_ob[0,0], err_op[0,0], appro_ob[0,0])

n =  7 (128, 128)
0.00286220803327422
0.0006709485731051255
n =  8 (256, 256)
0.0028623574552221406
0.0006716175211252401
n =  9 (512, 512)
0.0028626517531851192
0.0006725913518191965
n =  10 (1024, 1024)
0.002862755945093186
0.0006719952371922001


In [19]:
appro_U_18 = np.load(f'./data/exact_ob_9_1_18.npy')
appro_U_28 = np.load(f'./data/exact_ob_9_1_28.npy')
appro_U_85 = np.load(f'./data/exact_ob_9_1_85.npy')
appro_U_28 - appro_U_85

array([[ 1.10876450e-01+1.05853044e-01j,  1.30322901e-02-8.83995997e-04j,
         8.11128069e-03+2.21833848e-03j, ...,
         1.55045368e-09-1.72222480e-09j,  8.46805621e-10-1.83576306e-09j,
        -2.32676865e-10-1.14028224e-10j],
       [ 1.30322901e-02-8.83995981e-04j, -1.98717539e-02+1.14135515e-01j,
         7.77389272e-04-9.10797202e-04j, ...,
         5.47176067e-09+2.00686959e-08j, -2.32774082e-10-1.14855037e-10j,
         8.43216883e-10-1.83767636e-09j],
       [ 8.11663865e-03+2.19660448e-03j,  7.77389000e-04-9.10797126e-04j,
        -7.27762262e-02+3.25817607e-02j, ...,
        -1.37201516e-10-2.19998177e-10j,  5.43707044e-09+2.00871026e-08j,
         1.55395401e-09-1.72018619e-09j],
       ...,
       [ 1.55395401e-09-1.72018619e-09j,  5.43707044e-09+2.00871026e-08j,
        -1.37201516e-10-2.19998177e-10j, ...,
        -7.27762262e-02+3.25817607e-02j,  7.77389000e-04-9.10797126e-04j,
         8.11663865e-03+2.19660448e-03j],
       [ 8.43216883e-10-1.83767636e-09j, -2.

(0.9816722463159797-2.332325238166409e-17j) (-0.0012541762468709639-2.181573997780833e-17j) (0.9829264225628507-1.5075124038557586e-18j)

(0.9816722463159797-2.332325238166409e-17j) (-2.1008833811508865e-06-4.2249428514288504e-17j) (0.9816743471993609+1.8926176132624414e-17j)

0.0171
(0.9816722463159797-2.332325238166409e-17j) (0.000914037141718782-1.882305977087255e-17j) (0.9807582091742609-4.500192610791541e-18j)

In [ ]:
exact_ob_s - appro_ob_s

array([[-1.254176e-03-2.181574e-17j,  6.357182e-03-4.164055e-03j,
         1.916763e-05-1.260949e-05j, ...,  1.138114e-13+1.771224e-14j,
         2.546533e-14-2.143273e-15j, -3.808590e-24-1.113026e-14j],
       [ 6.357182e-03+4.164055e-03j,  1.250334e-03-1.912847e-17j,
        -3.839087e-04+1.884769e-03j, ..., -4.872551e-14-2.846347e-13j,
         6.187315e-24+1.218178e-15j, -2.546533e-14-2.143273e-15j],
       [ 1.916763e-05+1.260949e-05j, -3.839087e-04-1.884769e-03j,
        -1.263336e-03+4.499314e-17j, ...,  4.909774e-24+1.084614e-14j,
         4.872551e-14-2.846347e-13j, -1.138114e-13+1.771224e-14j],
       ...,
       [ 1.138114e-13-1.771224e-14j, -4.872551e-14+2.846347e-13j,
         4.909774e-24-1.084614e-14j, ...,  1.263336e-03+6.588282e-18j,
         3.839087e-04+1.884769e-03j, -1.916763e-05-1.260949e-05j],
       [ 2.546533e-14+2.143273e-15j,  6.187315e-24-1.218178e-15j,
         4.872551e-14+2.846347e-13j, ...,  3.839087e-04-1.884769e-03j,
        -1.250334e-03-3.024805e-18j

### Worst input (largest singular value)

In [ ]:
appro_U = scipy.linalg.expm(-1j * 1 * sum([h.to_matrix() for h in tfi.ham_parity]))
M = exact_U - appro_U
# np.linalg.eigh(M)
print('spectral norm by definition: ', sqrt(np.linalg.eigh(M.conj().T @ M)[0][-1]))
print('largest eigenval: ', np.linalg.eigh(M)[0][-1])
# np.linalg.norm(np.linalg.eigh(exact_U - appro_U)[1][-1], ord=2)
# np.linalg.norm(exact_U - appro_U, ord=-2)
U, S, V = np.linalg.svd(M)
print('largest singular value: ', S[0])
# print(V[0])
norm(V[0])
print(norm(M @ V[0].conj().T))
print(norm(M @ V[0]))
# print(norm(M @ np.linalg.eigh(M)[1][-1]).conj().T)
print('spectral norm by numpy: ', np.linalg.norm(M, ord=2))
# sorted(S)

spectral norm by definition:  (1.9998095531513846+0j)
largest eigenval:  2.8340126451426553
largest singular value:  1.9998095531513838
1.9998095531513842
1.9998095531513846
spectral norm by numpy:  1.9998095531513846


### l2 induced norm

In [ ]:
import scipy.linalg

t = 3.5
n = 6
h_test = sum([SparsePauliOp.from_sparse_list([(random.choices(['I','X','Y','Z'], k=n), random.sample(list(range(0, n)), n), 1)], n) for _ in range(5)])
h_test_I = h_test + SparsePauliOp('I'*n)
print(h_test, h_test_I)
np.linalg.norm(scipy.linalg.expm(-1j * t * h_test) - scipy.linalg.expm(-1j * t * h_test_I), ord=2)